## Problem Statement
Find the minimum of the parabola $F(x) = 10 * x^2$ using CMA.
We can encode an individual in the population as a vector, `(x)` of size `(1,)`. \
We can also choose some artifical bounds for `(x)` \
So, $(x) = (0)$ would be the ideal individual since we know that the ideal solution is $\begin{pmatrix} x \\ F(x)\end{pmatrix} = \begin{pmatrix} 0 \\ 0\end{pmatrix}$

In [14]:
import numpy as np
def generate_population(param_size, pop_size, low_bound, high_bound, discrete = False):
    """
    Generates a random matrix of size (pop_size, param_size) with values in the given bounds. 
    Integer values are used if discrete = True
    """
    population = np.random.uniform(low= low_bound, high = high_bound, size=(pop_size, param_size))
    if discrete:
        population = np.round(population)
    return population

In [15]:
PARAM_SIZE = 1
POP_SIZE = 10
LOW_BOUND = -10
HIGH_BOUND = 10
population = generate_population(PARAM_SIZE, POP_SIZE, LOW_BOUND, HIGH_BOUND)
print(population.shape)
print(np.mean(population))

(10, 1)
0.8150545323778731


In [16]:
def calculate_fitness(total_pop):
    """ Calculates fitness using the given function f(x) = 10 *x^2
    Returns a (POP_SIZE, ) numpy array
    """       
    fitness = 10 * np.square(total_pop)
    return fitness


In [18]:
fitness = calculate_fitness(population)
print(fitness.shape)
print(fitness.T)

(10, 1)
[[483.34727469 365.98104951 639.02251259  84.01740894  55.98184963
  109.49707636  80.57639583   8.52872461 194.90792406 502.30338264]]


In [64]:
def select_determinstic(t_pop, t_fitness, n_mating, n_params):
    """ Given current population t_pop, select n_mating number of parents using determinstic selection scheme
    based on t_fitness (the fitness of the population) 

    Returns parents and their fitness as a tuple (parent, fitness_of_parents)
    """    
    parents = np.empty((n_mating, n_params))
    
    best_fitness = np.argsort(t_fitness.flatten())[0:n_mating]
    parents = t_pop[best_fitness]
    return (parents, calculate_fitness(parents))

In [65]:
N_MATING = 4
parents, par_fitness = select_determinstic(population, fitness, N_MATING, PARAM_SIZE)
print(parents.shape)
print(parents)
print(par_fitness)

(4, 1)
[[ 0.92351094]
 [-2.36604839]
 [-2.83859817]
 [ 2.89857567]]
[[ 8.52872461]
 [55.98184963]
 [80.57639583]
 [84.01740894]]


In [79]:
def crossover(t_parents, n_offspring, idx_crossover):
    """ Given a set of parents, combine them and return offspring vectors
    
    Returns the offsprings and their fitness
    """
    
    # Create an emppty vector
    offspring = np.empty((n_offspring, t_parents.shape[1]))

    # Fill in crossover details
    for i in range(n_offspring):
        parent1 = t_parents[i]
        parent2 = t_parents[i+1]
        offspring[i] = np.copy(parent2)
        offspring[i,idx_crossover:] = parent1[idx_crossover:]
        
    return (offspring, calculate_fitness(offspring))
def mutation(t_offspring, mutation_rate):
    """ Given a set of offsprings, introduce mutation in them
   
    Returns the mutated offsprings and their fitness
    """
    # Fill in details
    mutated_offspring = np.copy(t_offspring)
    
    random_values = np.random.uniform(low=-0.5, high=0.5, size=t_offspring.shape)
    coin_toss = np.random.uniform(low=0, high=1, size=t_offspring.shape)
    ind = np.nonzero(coin_toss > mutation_rate)
    print(ind)
    print(random_values.shape)
    mutated_offspring[ind] += random_values[ind]
        
    return (mutated_offspring, calculate_fitness(mutated_offspring))

In [87]:
N_OFFSPRING =  N_MATING-1 # Fill in how many offspring you want
IDX_CROSSOVER = 0 # Fill in at which index you want crossover
PM = 0.5 # Mutation rate parameter

offspring,  offspring_fitness = crossover(parents, N_OFFSPRING, IDX_CROSSOVER)
offspring, offspring_fitness = mutation(offspring, 0.5)
print(offspring)
print(offspring_fitness)

(array([0, 1]), array([0, 0]))
(3, 1)
[[ 0.92674444]
 [-2.78891994]
 [-2.83859817]]
[[ 8.58855266]
 [77.78074407]
 [80.57639583]]


In [95]:
def environmental_selection(curr_population, offspring):
    """ Calculate total population (after constraint checking) fitness,
    rank accoridingly and select only the top POP_SIZE individuals to
    pass on to the next generation
    """ 

    # Fill in details
    t_total_pop = np.vstack((curr_population, offspring))
    new_pop, new_pop_fitness = select_determinstic(t_total_pop, calculate_fitness(t_total_pop), curr_population.shape[0], curr_population.shape[1])
    return new_pop

In [96]:
new_pop = environmental_selection(population, offspring)
print(new_pop)

[[ 0.92351094]
 [ 0.92674444]
 [-2.36604839]
 [-2.78891994]
 [-2.83859817]
 [-2.83859817]
 [ 2.89857567]
 [ 3.30903425]
 [ 4.41483776]
 [-6.04963676]]


In [99]:
best_outputs = []
num_generations = 1000
curr_population = generate_population(PARAM_SIZE, POP_SIZE, LOW_BOUND, HIGH_BOUND)
overall_max_fitness = 99999

# Run many iterations
# You should also have another convergence check
for generation in range(num_generations):
    print("Generation : ", generation)

    # Measuring the fitness of each chromosome in the population.
    fitness = calculate_fitness(curr_population)
    print('Fitness')
    print(fitness.shape)
    print(fitness.T)

    min_fitness = np.min(fitness)

    # The best result in the current iteration.
    print("Best result in current iteration {0} compared to overall {1}".format(min_fitness, min(min_fitness, overall_max_fitness)))
    best_outputs.append(min_fitness)
    
    # Selecting the best parents in the population for mating.
    N_MATING = 6
    parents, par_fitness = select_determinstic(curr_population, fitness, N_MATING, PARAM_SIZE)
    print("Parents")
    print(parents.shape)
    print(parents)
    print(par_fitness)
#     parents, _ = select_stochastic(curr_population, fitness)
    
    # print("Parents")
    # print(parents)
    N_OFFSPRING =  N_MATING-1 # Fill in how many offspring you want
    IDX_CROSSOVER = 0 # Fill in at which index you want crossover
    PM = 0.5 # Mutation rate parameter

    offspring,  offspring_fitness = crossover(parents, N_OFFSPRING, IDX_CROSSOVER)
    offspring, offspring_fitness = mutation(offspring, PM)
    print("Mutation")
    print(offspring)
    print(offspring_fitness)

    # print("Mutation")
    # print(offspring_mutation)

    # Environmental selection
    curr_population = environmental_selection(curr_population, offspring)
              
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = calculate_fitness(curr_population)
print(fitness)
# Then return the index of that solution corresponding to the best fitness.
max_idx = np.argmin(fitness)

print("Best solution : ", curr_population[max_idx, :])
print("Best solution fitness : ", fitness[max_idx])

Generation :  0
Fitness
(10, 1)
[[400.12448372 487.63050396 104.95019615 129.14716767 294.74954932
  674.94718622 393.81801126  34.7962367   66.79147559 538.65828   ]]
Best result in current iteration 34.79623669930828 compared to overall 34.79623669930828
Parents
(6, 1)
[[ 1.86537494]
 [ 2.58440468]
 [-3.23960177]
 [ 3.59370516]
 [-5.42908417]
 [-6.2754921 ]]
[[ 34.7962367 ]
 [ 66.79147559]
 [104.95019615]
 [129.14716767]
 [294.74954932]
 [393.81801126]]
(array([0, 2]), array([0, 0]))
(5, 1)
Mutation
[[ 2.28470309]
 [ 2.58440468]
 [-3.58607006]
 [ 3.59370516]
 [-5.42908417]]
[[ 52.19868223]
 [ 66.79147559]
 [128.59898457]
 [129.14716767]
 [294.74954932]]
Generation :  1
Fitness
(10, 1)
[[ 34.7962367   52.19868223  66.79147559  66.79147559 104.95019615
  128.59898457 129.14716767 129.14716767 294.74954932 294.74954932]]
Best result in current iteration 34.79623669930828 compared to overall 34.79623669930828
Parents
(6, 1)
[[ 1.86537494]
 [ 2.28470309]
 [ 2.58440468]
 [ 2.58440468]
 [-3